# **EECS 492 HW4**

#**Setup**

When running this cell, give this script permission so that you can mount this Colab notebook to your Google Drive.:


In [2]:
from google.colab import drive
from google.colab import runtime
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%pwd

'/content'

In [4]:
%cd gdrive/MyDrive/

/content/gdrive/MyDrive


Clone the EECS 492 Git Repository so that we can grab the necessary files to work with! To see where this is in our Google Colab directory, click the folder icon on the left side of the notebook.

In [5]:
!git clone https://github.com/lialex123/eecs492hw4.git

fatal: destination path 'eecs492hw4' already exists and is not an empty directory.


In [6]:
%cd eecs492hw4

/content/gdrive/MyDrive/eecs492hw4


In [ ]:
%cd ..

/content/gdrive



Run the following code to import the modules you'll need. 

In [7]:
!pip3 install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 1.9 MB/s 
     |████████████████████████████████| 86 kB 3.4 MB/s 
     |████████████████████████████████| 750.6 MB 9.6 kB/s 
     |████████████████████████████████| 78 kB 5.2 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

This cell will load all the necessary libraries and set a seed so we can run the algorithm deterministically for grading

In [25]:
import logging

import coloredlogs

from Coach import Coach
from othello.OthelloGame import OthelloGame as Game
from othello.pytorch.NNet import NNetWrapper as nn
from utils import *

import torch
import random
import numpy as np

from IPython.display import clear_output
import time

seed = 492
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

This cell specifies the hyperparameters used for the algorithm. Don't modify the hyperparameters until you have recieved all points on GradesScope. After that, feel free to optimize the hyperparameters (this is completely optional)!

In [26]:
log = logging.getLogger(__name__)

coloredlogs.install(level='INFO')  # Change this to DEBUG to see more info.

# the only hyperparameters I modified from the original are numIters, numEps, and arenaCompare
# feel free to tweak them once your code is working
# original values for reference: numIters = 1000, numEps = 100, arenaCompare = 25
args = dotdict({
    'numIters': 1,
    'numEps': 2,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 5,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 5,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,

    'checkpoint': './temp/',
    'load_model': False,
    'load_folder_file': ('/dev/models/8x100x50','best.pth.tar'),
    'numItersForTrainExamplesHistory': 20,

})

This cell initializes all the necessary objects to start training

In [27]:
log.info('Loading %s...', Game.__name__)
g = Game(6)

log.info('Loading %s...', nn.__name__)
nnet = nn(g)

if args.load_model:
    log.info('Loading checkpoint "%s/%s"...', args.load_folder_file[0], args.load_folder_file[1])
    nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])
else:
    log.warning('Not loading a checkpoint!')

log.info('Loading the Coach...')
c = Coach(g, nnet, args)

if args.load_model:
    log.info("Loading 'trainExamples' from file...")
    c.loadTrainExamples()

2022-11-18 05:08:55 53178a7e3449 __main__[75] INFO Loading OthelloGame...
2022-11-18 05:08:55 53178a7e3449 __main__[75] INFO Loading NNetWrapper...
2022-11-18 05:08:55 53178a7e3449 __main__[75] WARNING Not loading a checkpoint!
2022-11-18 05:08:55 53178a7e3449 __main__[75] INFO Loading the Coach...


Starts the training process

In [28]:
log.info('Starting the learning process 🎉')
c.learn()

2022-11-18 05:08:58 53178a7e3449 __main__[75] INFO Starting the learning process 🎉
2022-11-18 05:08:58 53178a7e3449 Coach[75] INFO Starting Iter #1 ...
Self Play:   0%|          | 0/2 [00:00<?, ?it/s]


NameError: ignored

Once the algorithm is done training, print out the actions the algorithm took in the selection phase of MCTS. The output from this should match the instructor's.

In [ ]:
instrResults = [[(2, 1), (3, 1), (4, 2), (5, 1), (4, 1), (3, 0), (4, 0), (1, 1), (1, 3), (1, 2), (0, 1), (1, 0), (5, 2), (0, 3), (2, 0), (0, 0), (2, 4), (0, 2), (0, 4), (1, 5), (2, 5), (5, 3), (4, 4), (3, 4), (0, 5), (1, 4), (3, 5), (4, 5), (5, 5), (5, 4), (5, 0), (4, 3), (3, 4), (2, 4), (1, 2), (2, 1), (3, 0), (4, 4), (3, 5), (4, 1), (1, 3), (2, 0), (3, 1), (2, 5), (5, 2), (0, 3), (0, 4), (4, 0), (5, 4), (0, 5), (0, 2), (4, 2), (1, 4), (4, 5), (1, 0), (1, 5), (5, 0), (1, 1), (0, 1), (5, 3), (4, 3), (0, 0), (5, 5), (5, 1)]]
print(c.actionsTaken)
print("Your actions match the instructor's:", instrResults == c.actionsTaken)

Below is a visualization of the actions you took in the first iteration of the model. Feel free to run it if you want to see how your algorithm did! Modify if you want to run a different iteration (only applies if you changed the hyperparamers).

In [ ]:
iteration = 0
board = g.getInitBoard()

g.display(board)
for i, move in enumerate(c.actionsTaken[iteration]):
  # converts (x, y) to an action the game understands
  action = move[0]*g.n + move[1]
  board = g.getNextState(board, 1 if i % 2 == 0 else -1, action)[0]

  time.sleep(2)
  clear_output(wait=False)
  g.display(board)

print("game finished!")